# Problem 56: Powerful digit sum
   
A googol ($10^{100}$) is a massive number: one followed by one-hundred zeros; $100^{100}$ is almost unimaginably large: one followed by two-hundred zeros. Despite their size, the sum of the digits in each number is only 1.

Considering natural numbers of the form, $a^b$, where $a, b < 100$, what is the maximum digital sum?

In [1]:
def power_digit_sum(n):
    '''
    Maximum digital sum for integers of the form
    a^b where a, b < n
    '''
    # check digit sum for all a, b pairs
    m = 0
    for a in range(n):
        for b in range(n):
            ps = sum([int(d) for d in str(a**b)])
            if ps > m:
                m = ps
    return m

In [2]:
power_digit_sum(100)
# should return 972

972

# Problem 57: Square root convergents
It is possible to show that the square root of two can be expressed as an infinite continued fraction.

$\sqrt{2} = 1 + \dfrac{1}{2 + \frac{1}{2 + \frac{1}{2 + ...}}}$

By expanding this for the first four iterations, we get:

$1 + \frac{1}{2} = \frac{3}{2} = 1.5$

$1 + \frac{1}{2+\frac{1}{2}} = \frac{7}{5} = 1.4$

$1 + \frac{1}{2+\frac{1}{2+\frac{1}{2}}} = \frac{17}{12} = 1.41666...$

$1 + \frac{1}{2+\frac{1}{2+\frac{1}{2+\frac{1}{2}}}} = \frac{41}{29} = 1.41379...$


The next three expansions are $\frac{99}{70}$, $\frac{239}{169}$, and $\frac{577}{408}$ but the eighth expansion, $\frac{1393}{985}$, is the first example where the number of digits in the numerator exceeds the number of digits in the denominator.

In the first one-thousand expansions, how many fractions contain a numerator with more digits than the denominator?

In [3]:
from math import gcd

In [4]:
def euler57(iters):
    '''
    Number of expansions of root 2 in which the numerator contains 
    more digits than the denominator (within iters iterations)
    '''
    if iters < 8:
        return 0
    
    # iteratively calculate numerator and denominator
    tot = 0
    num, den = 3, 2
    for i in range(iters-1):
        num, den = (num + 2*den), (num + den)
        num, den = (num // gcd(num,den)), (den // gcd(num, den))
        if len(str(num)) > len((str(den))):
            tot += 1
    return tot

In [5]:
euler57(1000)
# should return 153

153

# Problem 58: Spiral primes

Starting with 1 and spiralling anticlockwise in the following way, a square spiral with side length 7 is formed.

$\underline{\textbf{37}} \ 36 \ 35 \ 34 \ 33 \ 32 \ \underline{\textbf{31}}$

$38 \ \underline{\textbf{17}} \ 16 \ 15 \ 14 \ \underline{\textbf{13}} \ 30$

$39 \ 18 \ \underline{\textbf{05}} \ 04 \ \underline{\textbf{03}} \ 12 \ 29$

$40 \ 19 \ 06 \ 01 \ 02 \ 11 \ 28 $

$41 \ 20 \ \underline{\textbf{07}} \ 08 \ 09 \ 10 \ 27$

$42 \ 21 \ 22 \ 23 \ 24 \ 25 \ 26$

$\underline{\textbf{43}} \ 44 \ 45 \ 46 \ 47 \ 48 \ 49$

It is interesting to note that the odd squares lie along the bottom right diagonal, but what is more interesting is that 8 out of the 13 numbers lying along both diagonals are prime; that is, a ratio of $8/13 \approx 62\%$.

If one complete new layer is wrapped around the spiral above, a square spiral with side length 9 will be formed. If this process is continued, what is the side length of the square spiral for which the ratio of primes along both diagonals first falls below 10%?

In [6]:
from itertools import count

In [7]:
def p(N):
    '''
    True if N prime, else false
    '''
    if N < 2: return False 
    root = int(N**0.5)
    for i in range(2,root+1):
        if (N%i == 0):
            return False
    return True

In [8]:
def spiral_primes(l_max, threshold):
    '''
    Side length of number spiral (below l_max) at which fraction
    of numbers along diagonal are prime is > threshold
    '''
    if l_max < 3 and threshold > 0: return 1
    # iterate over odd side lengths
    n_prime, n_diag = 0, 1
    for l in range(3, l_max+1, 2):
        n_diag += 4
        N = (l-2)**2 + l - 1
        # check how many corners are prime and add
        n_prime += p(N) + p(N+l-1) + p(N+2*(l-1))
        if (n_prime / n_diag) < threshold:
            return l
    print('No solutions found within range')
    return

In [9]:
spiral_primes(10**5, 0.1)
# should return 26241

26241

# Problem 59: XOR decryption
   
Each character on a computer is assigned a unique code and the preferred standard is ASCII (American Standard Code for Information Interchange). For example, uppercase A = 65, asterisk (*) = 42, and lowercase k = 107.

A modern encryption method is to take a text file, convert the bytes to ASCII, then XOR each byte with a given value, taken from a secret key. The advantage with the XOR function is that using the same encryption key on the cipher text, restores the plain text; for example, 65 XOR 42 = 107, then 107 XOR 42 = 65.

For unbreakable encryption, the key is the same length as the plain text message, and the key is made up of random bytes. The user would keep the encrypted message and the encryption key in different locations, and without both "halves", it is impossible to decrypt the message.

Unfortunately, this method is impractical for most users, so the modified method is to use a password as a key. If the password is shorter than the message, which is likely, the key is repeated cyclically throughout the message. The balance for this method is using a sufficiently long password key for security, but short enough to be memorable.

Your task has been made easy, as the encryption key consists of three lower case characters. Using data > p059_cipher.txt, a file containing the encrypted ASCII codes, and the knowledge that the plain text must contain common English words, decrypt the message and find the sum of the ASCII values in the original text.

In [10]:
import numpy as np
from itertools import product

In [11]:
# load cipher
from pathlib import Path
cipher_file = Path('data') / 'p059_cipher.txt'
cipher = np.loadtxt(cipher_file, delimiter=',', dtype=int)

In [12]:
lower_ords = [i for i in range(ord('a'), ord('z')+1)]
upper_ords = [i for i in range(ord('A'), ord('Z')+1)]
num_ords = [i for i in range(ord('0'), ord('9')+1)]
space_ords = [i for i in range(33)]
punct_ords = [33,34,38,39,40,41,44,45,46,47,58,59,63,96]
misc = [ord('+'), ord('['), ord(']')]
allowed_ords = lower_ords + upper_ords + num_ords + \
                space_ords + punct_ords + misc

In [13]:
def decrypt(cipher):
    '''
    Find keys of 3 lowercase letters which decrypt
    ASCII values in cipher to produce string of 
    common letters, numbers and punctuation, and sum
    decrypted ASCII values
    '''
    len_key = 3
    
    # for each position within key find set of candidates that
    # give common symbols for each relevant byte in cipher
    candidates = []
    for pos in range(len_key):
        candidates += [[]]
        for lord in lower_ords:
            for c in cipher[pos::len_key]:
                if c^lord not in allowed_ords:
                    break
            else:
                candidates[pos] += [lord]
                
    # print sum and decrypted message for every possible key
    for combo in product(*candidates):
        tot = 0
        decrypted = ''
        for i, c in enumerate(cipher):
            pos = i % len_key
            xor = c ^ combo[pos]
            tot += xor
            decrypted += chr(xor)
        print('sum = %d\n' % tot)
        print('Decrypted text:\n\n%s' % decrypted)

In [14]:
decrypt(cipher)
# should return sum of 129448

sum = 129448

Decrypted text:

An extract taken from the introduction of one of Euler's most celebrated papers, "De summis serierum reciprocarum" [On the sums of series of reciprocals]: I have recently found, quite unexpectedly, an elegant expression for the entire sum of this series 1 + 1/4 + 1/9 + 1/16 + etc., which depends on the quadrature of the circle, so that if the true sum of this series is obtained, from it at once the quadrature of the circle follows. Namely, I have found that the sum of this series is a sixth part of the square of the perimeter of the circle whose diameter is 1; or by putting the sum of this series equal to s, it has the ratio sqrt(6) multiplied by s to 1 of the perimeter to the diameter. I will soon show that the sum of this series to be approximately 1.644934066842264364; and from multiplying this number by six, and then taking the square root, the number 3.141592653589793238 is indeed produced, which expresses the perimeter of a circle whose diameter is 

# Problem 60: Prime pair sets

The primes 3, 7, 109, and 673, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking 7 and 109, both 7109 and 1097 are prime. The sum of these four primes, 792, represents the lowest sum for a set of four primes with this property.

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

In [15]:
import numpy as np

In [16]:
def P(N):
    '''
    True if N prime, else false
    '''
    if N < 2: return False 
    root = int(N**0.5)
    for i in range(2,root+1):
        if (N%i == 0):
            return False
    return True

In [17]:
def prime_pair_set(size, threshold):
    '''
    Lowest sum of set of primes in which concatenation of
    any two primes yields a prime, the size of the set equals 
    size and sum is <= threshold
    '''
    # iterate over odd N (2 < N < threshold)
    primes, prime_sets, prime_sums = [], [], []
    N = 3
    opt_set, lowest_sum = [], np.inf
    while N < threshold:
        # check N is prime
        is_prime = True
        root = N**0.5
        for p in primes:
            if p > root:
                break
            if N%p == 0:
                is_prime = False
                break
                
        if not is_prime:
            N += 2
            continue    
        primes += [N]
        
        # find sets in which N forms pair with all members
        sN = str(N)
        checked = {}
        for i in range(len(prime_sets)):
            pset, psum = prime_sets[i], prime_sums[i]
            l = len(pset)
            
            # can skip if set larger than required,
            # or if sum of potential set is > threshold 
            if l == size or (N*(size-l)+psum) > threshold:
                continue
                
            for sp in pset:
                # check if N forms pair with prime
                if sp in checked:
                    # pair already checked
                    conc_prime = checked[sp]
                else:
                    Np, pN = int(sp + sN), int(sN + sp)
                    conc_prime = P(Np) and P(pN)
                    checked[sp] = conc_prime    
                if not conc_prime:
                    break
            else:
                # add to prime sets, compare sum to current best
                new_set, new_sum = (pset + [sN]), (psum + N)
                prime_sets += [new_set]
                prime_sums += [new_sum]
                if (l + 1) == size:
                    print('%s, sum = %d' %(new_set, new_sum))
                    if new_sum < lowest_sum:
                        opt_set, lowest_sum = new_set, new_sum
                        threshold = min(threshold, new_sum)           
        prime_sets += [[sN]]
        prime_sums += [N]
        N += 2
    if opt_set:
        return lowest_sum
    print('No solutions found for primes under threshold')
    return

In [18]:
prime_pair_set(5, 30000)
# should return 26033

['13', '5197', '5701', '6733', '8389'], sum = 26033


26033